In [1]:
import pandas as pd

In [10]:
# Load data
file_path = "data/redlat.genes.chr.counts.hg38_multianno.tsv"
original_df = pd.read_csv(file_path, sep='\t')

# Filter data we care about 
df = original_df[
    (original_df["Func.refGene"] == "exonic") & 
    (original_df["ExonicFunc.refGene"] == "nonsynonymous SNV")
]

In [ ]:
# Verify filtering worked
df["AAChange.refGene"].head()

45       TARDBP:NM_007375:exon2:c.G149C:p.C50S
323     TARDBP:NM_007375:exon6:c.A776G:p.N259S
324     TARDBP:NM_007375:exon6:c.G881T:p.G294V
326    TARDBP:NM_007375:exon6:c.A1147G:p.I383V
327    TARDBP:NM_007375:exon6:c.G1154T:p.W385L
Name: AAChange.refGene, dtype: object

In [25]:
desired_isoforms = {
    "PSEN1": "NM_000021",
    "PSEN2": "NM_000447",
    "TARDBP": "NM_007375",
}

In [35]:
# Let's parse all the rows and select the correct variant name
# It will be in the AAChange.refGene column, but there is a lot of fields to parse

def extract_variant(row):
    isoforms = row["AAChange.refGene"].split(",")
    all_affected = row["All_affected"]
    all_unaffected = row["All_unaffected"]

    case_cnt = int(all_affected.split("/")[1])
    control_cnt = int(all_unaffected.split("/")[1])
    
    for isoform in isoforms:
        parts = isoform.split(":")
        if len(parts) < 4:
            continue  # Skip malformed entries

        gene_name, isoform_id = parts[0], parts[1]

        if gene_name not in desired_isoforms:
            continue  # Skip unknown gene names

        if desired_isoforms[gene_name] == isoform_id:
            aavariant = parts[-1][2:]  # Extract amino acid variant
            aa_number = aavariant[1:-1] if len(aavariant) > 2 else ""  # Extract the inner number
            return pd.Series([aavariant, aa_number, case_cnt, control_cnt])

    return pd.Series([None, None, case_cnt, control_cnt])  # Return None if no match found

df[["variant", "AA", "case", "control"]] = df.apply(extract_variant, axis=1)
df["case"] = df["case"].astype(int)
df["control"] = df["control"].astype(int)


/var/folders/5n/gkdqz59j4x5c3d4g045zn_n40000gn/T/ipykernel_20980/3946025148.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["variant", "AA", "case", "control"]] = df.apply(extract_variant, axis=1)
/var/folders/5n/gkdqz59j4x5c3d4g045zn_n40000gn/T/ipykernel_20980/3946025148.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["case"] = df["case"].astype(int)
/var/folders/5n/gkdqz59j4x5c3d4g045zn_n40000gn/T/ipykernel_20980/3946025148.py:31: SettingWithCopyWarning: 
A value is trying to be set on a 

In [36]:
# Now export the selected columns
df[["AA", "variant", "case", "control"]].to_csv("output.csv", index=False)